In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load data
match_df = pd.read_csv('../data/raw/Match.csv')
team_df = pd.read_csv('../data/raw/Team.csv')

# Convert date to datetime
match_df['date'] = pd.to_datetime(match_df['date'])

# Get all columns for home/draw/away odds
home_cols = [col for col in match_df.columns if col.endswith('H')]
draw_cols = [col for col in match_df.columns if col.endswith('D')]
away_cols = [col for col in match_df.columns if col.endswith('A')]

# Drop rows where all odds are missing
match_df.dropna(subset=home_cols + draw_cols + away_cols, how='all', inplace=True)

# Calculate average odds across all sources
match_df['Avg_Home_Odds'] = match_df[home_cols].mean(axis=1)
match_df['Avg_Draw_Odds'] = match_df[draw_cols].mean(axis=1)
match_df['Avg_Away_Odds'] = match_df[away_cols].mean(axis=1)

# Add score column (format: "Home Goals - Away Goals")
match_df['Score'] = match_df['home_team_goal'].astype(str) + ' - ' + match_df['away_team_goal'].astype(str)

# Add Venue column (from the perspective of the home team)
match_df['Venue'] = 'Home'

# Create a dictionary for team_api_id to team_long_name mapping
team_dict = team_df.set_index('team_api_id')['team_long_name'].to_dict()

# Add Opponent column
match_df['Opponent'] = match_df['away_team_api_id'].map(team_dict)

# Prepare features and targets
# Using average odds as features for simplicity
features = ['Avg_Home_Odds', 'Avg_Draw_Odds', 'Avg_Away_Odds']
target_home = 'home_team_goal'
target_away = 'away_team_goal'

# Split the entire dataset into training and testing sets (1/3 for training)
train_df, test_df = train_test_split(match_df, test_size=1/3, random_state=42)

# Train models for home and away goals
model_home = RandomForestRegressor(n_estimators=100, random_state=42)
model_away = RandomForestRegressor(n_estimators=100, random_state=42)

model_home.fit(train_df[features], train_df[target_home])
model_away.fit(train_df[features], train_df[target_away])

print("Model trained on one-third of the entire dataset.")

Model trained on one-third of the entire dataset.


In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

# Assuming 'match_df', 'team_df', 'model_home', 'model_away', and 'team_dict' are available from Cell Block 1

# Prompt user to choose a team and season
team_name = input("Enter the team name: ")
start_date = input("Enter the start date of the season (YYYY-MM-DD): ")
end_date = input("Enter the end date of the season (YYYY-MM-DD): ")

# Get the team_api_id for the selected team
team_id = team_df[team_df['team_long_name'] == team_name]['team_api_id'].values[0]

# Filter matches for the selected team within the date range
team_matches = match_df[
    ((match_df['home_team_api_id'] == team_id) | (match_df['away_team_api_id'] == team_id)) &
    (match_df['date'] >= start_date) & (match_df['date'] <= end_date)
].copy()

# Add Venue column from the team's perspective
team_matches['Venue'] = team_matches.apply(
    lambda row: 'Home' if row['home_team_api_id'] == team_id else 'Away', axis=1
)

# Add Opponent column
team_matches['Opponent'] = team_matches.apply(
    lambda row: team_dict.get(row['away_team_api_id']) if row['home_team_api_id'] == team_id 
    else team_dict.get(row['home_team_api_id']), axis=1
)

# Prepare features for prediction
X_team = team_matches[features]

# Predict home and away goals
home_pred = np.round(model_home.predict(X_team)).astype(int)
away_pred = np.round(model_away.predict(X_team)).astype(int)

# Adjust predictions based on the team's venue
team_matches['Predicted_Team_Goals'] = team_matches.apply(
    lambda row: home_pred[team_matches.index.get_loc(row.name)] if row['Venue'] == 'Home' 
    else away_pred[team_matches.index.get_loc(row.name)], axis=1
)
team_matches['Predicted_Opponent_Goals'] = team_matches.apply(
    lambda row: away_pred[team_matches.index.get_loc(row.name)] if row['Venue'] == 'Home' 
    else home_pred[team_matches.index.get_loc(row.name)], axis=1
)

# Actual goals
team_matches['Actual_Team_Goals'] = team_matches.apply(
    lambda row: row['home_team_goal'] if row['Venue'] == 'Home' else row['away_team_goal'], axis=1
)
team_matches['Actual_Opponent_Goals'] = team_matches.apply(
    lambda row: row['away_team_goal'] if row['Venue'] == 'Home' else row['home_team_goal'], axis=1
)

# Function to determine outcome
def get_outcome(team_goals, opp_goals):
    if team_goals > opp_goals:
        return 'Win'
    elif team_goals == opp_goals:
        return 'Draw'
    else:
        return 'Loss'

# Predicted and actual outcomes
team_matches['Predicted_Outcome'] = team_matches.apply(
    lambda row: get_outcome(row['Predicted_Team_Goals'], row['Predicted_Opponent_Goals']), axis=1
)
team_matches['Actual_Outcome'] = team_matches.apply(
    lambda row: get_outcome(row['Actual_Team_Goals'], row['Actual_Opponent_Goals']), axis=1
)

# Create predictions table
predictions_table = team_matches[['date', 'Opponent', 'Venue', 'Predicted_Team_Goals', 'Predicted_Opponent_Goals', 'Predicted_Outcome']]
predictions_table['Predicted_Score'] = predictions_table['Predicted_Team_Goals'].astype(str) + ' - ' + predictions_table['Predicted_Opponent_Goals'].astype(str)

# Create actual results table
actual_table = team_matches[['date', 'Opponent', 'Venue', 'Actual_Team_Goals', 'Actual_Opponent_Goals', 'Actual_Outcome']]
actual_table['Actual_Score'] = actual_table['Actual_Team_Goals'].astype(str) + ' - ' + actual_table['Actual_Opponent_Goals'].astype(str)

# Sort both tables by date
predictions_table = predictions_table.sort_values(by='date').reset_index(drop=True)
actual_table = actual_table.sort_values(by='date').reset_index(drop=True)

# Display the tables
print(f"\n=== Predictions for {team_name} from {start_date} to {end_date} ===")
display(predictions_table[['date', 'Opponent', 'Venue', 'Predicted_Score', 'Predicted_Outcome']])

print(f"\n=== Actual Results for {team_name} from {start_date} to {end_date} ===")
display(actual_table[['date', 'Opponent', 'Venue', 'Actual_Score', 'Actual_Outcome']])


=== Predictions for Real Madrid CF from 2014/08/08 to 2015/05/23 ===


/var/folders/fn/g4kn2wts7y197l6m0pqplkhm0000gn/T/ipykernel_49317/345139426.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_table['Predicted_Score'] = predictions_table['Predicted_Team_Goals'].astype(str) + ' - ' + predictions_table['Predicted_Opponent_Goals'].astype(str)
/var/folders/fn/g4kn2wts7y197l6m0pqplkhm0000gn/T/ipykernel_49317/345139426.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_table['Actual_Score'] = actual_table['Actual_Team_Goals'].astype(str) + ' - ' + actual_t

,date,Opponent,Venue,Predicted_Score,Predicted_Outcome
0,2014-08-25,Córdoba CF,Home,2 - 0,Win
1,2014-08-31,Real Sociedad,Away,2 - 3,Loss
2,2014-09-13,Atlético Madrid,Home,2 - 1,Win
3,2014-09-20,RC Deportivo de La Coruña,Away,3 - 1,Win
4,2014-09-23,Elche CF,Home,4 - 0,Win
5,2014-09-27,Villarreal CF,Away,2 - 0,Win
6,2014-10-05,Athletic Club de Bilbao,Home,4 - 0,Win
7,2014-10-18,Levante UD,Away,3 - 0,Win
8,2014-10-25,FC Barcelona,Home,3 - 1,Win
9,2014-11-01,Granada CF,Away,3 - 0,Win



=== Actual Results for Real Madrid CF from 2014/08/08 to 2015/05/23 ===


,date,Opponent,Venue,Actual_Score,Actual_Outcome
0,2014-08-25,Córdoba CF,Home,2 - 0,Win
1,2014-08-31,Real Sociedad,Away,2 - 4,Loss
2,2014-09-13,Atlético Madrid,Home,1 - 2,Loss
3,2014-09-20,RC Deportivo de La Coruña,Away,8 - 2,Win
4,2014-09-23,Elche CF,Home,5 - 1,Win
5,2014-09-27,Villarreal CF,Away,2 - 0,Win
6,2014-10-05,Athletic Club de Bilbao,Home,5 - 0,Win
7,2014-10-18,Levante UD,Away,5 - 0,Win
8,2014-10-25,FC Barcelona,Home,3 - 1,Win
9,2014-11-01,Granada CF,Away,4 - 0,Win


In [3]:
# Calculate statistics
num_matches = len(team_matches)
actual_avg_team_goals = team_matches['Actual_Team_Goals'].mean()
actual_avg_opp_goals = team_matches['Actual_Opponent_Goals'].mean()
actual_record = team_matches['Actual_Outcome'].value_counts()
predicted_avg_team_goals = team_matches['Predicted_Team_Goals'].mean()
predicted_avg_opp_goals = team_matches['Predicted_Opponent_Goals'].mean()
predicted_record = team_matches['Predicted_Outcome'].value_counts()

# Print statistics
print(f"\n=== Statistics for {team_name} from {start_date} to {end_date} ({num_matches} matches) ===")
print("\n-- Actual --")
print(f"Average Goals Scored by Team: {actual_avg_team_goals:.2f}")
print(f"Average Goals Conceded: {actual_avg_opp_goals:.2f}")
print(f"Record (W-D-L): {actual_record.get('Win', 0)}-{actual_record.get('Draw', 0)}-{actual_record.get('Loss', 0)}")
print("\n-- Predicted --")
print(f"Average Goals Scored by Team: {predicted_avg_team_goals:.2f}")
print(f"Average Goals Conceded: {predicted_avg_opp_goals:.2f}")
print(f"Record (W-D-L): {predicted_record.get('Win', 0)}-{predicted_record.get('Draw', 0)}-{predicted_record.get('Loss', 0)}")


=== Statistics for Real Madrid CF from 2014/08/08 to 2015/05/23 (38 matches) ===

-- Actual --
Average Goals Scored by Team: 3.11
Average Goals Conceded: 1.00
Record (W-D-L): 30-2-6

-- Predicted --
Average Goals Scored by Team: 2.71
Average Goals Conceded: 0.84
Record (W-D-L): 34-0-4
